#Multilabel multiclass classification

(src: https://github.com/DhavalTaunk08/NLP_scripts/blob/master/Transformers_multilabel_distilbert.ipynb)

Using the Jigsaw toxic data from Kaggle (https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

The data contains the text, and the labels using one hot encoding

You are provided with a large number of Wikipedia comments which have been labeled by human raters for toxic behavior. The types of toxicity are:

* toxic
* severe_toxic
* obscene
* threat
* insult
* identity_hate

You must create a model which predicts a probability of each type of toxicity for each comment.



In [1]:
! pip install transformers

In [1]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
print(device)

cuda


In [4]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [5]:
data = pd.read_csv('final_dataset.tsv', sep='\t')

In [6]:
data.columns

Index(['PMID', 'Title/Abstract', 'MeshTerms', 'SemanticTypes'], dtype='object')

In [7]:
data.head()

,PMID,Title/Abstract,MeshTerms,SemanticTypes
0,34694464,limit use dithionit quench determin topolog me...,D005456;D008565,T130;T192;T116;T123
1,34694463,isol post traumat astereognosi case base revie...,D006225;D013236,T039;T023
2,34694462,loss chromatin remodel ddm caus segreg distort...,D017360;D029681,T116;T002
3,34694461,identif novel genom wide pleiotrop associ oral...,D058685,T028;T045
4,34694460,rare frequent lethal complic pulmonari vein is...,D001281;D017115;D004937;D011667,T023;T047;T046;T190;T061


In [8]:
data.drop(['PMID'], inplace=True, axis=1)
data.drop(['MeshTerms'], inplace=True, axis=1)
data.head()

,Title/Abstract,SemanticTypes
0,limit use dithionit quench determin topolog me...,T130;T192;T116;T123
1,isol post traumat astereognosi case base revie...,T039;T023
2,loss chromatin remodel ddm caus segreg distort...,T116;T002
3,identif novel genom wide pleiotrop associ oral...,T028;T045
4,rare frequent lethal complic pulmonari vein is...,T023;T047;T046;T190;T061


In [9]:
df_onehot = pd.concat([data.drop('SemanticTypes', 1), data['SemanticTypes'].str.get_dummies(sep=";")], 1)
df_onehot.head()

,Title/Abstract,T001,T002,T004,T005,T007,T008,T010,T011,T012,...,T190,T191,T192,T194,T195,T196,T197,T200,T201,T204
0,limit use dithionit quench determin topolog me...,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,isol post traumat astereognosi case base revie...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,loss chromatin remodel ddm caus segreg distort...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,identif novel genom wide pleiotrop associ oral...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,rare frequent lethal complic pulmonari vein is...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [10]:
new_df = pd.DataFrame()
new_df['text'] = df_onehot['Title/Abstract']
new_df['labels'] = df_onehot.iloc[:, 1:].values.tolist()

In [11]:
new_df.head()

,text,labels
0,limit use dithionit quench determin topolog me...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,isol post traumat astereognosi case base revie...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,loss chromatin remodel ddm caus segreg distort...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,identif novel genom wide pleiotrop associ oral...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,rare frequent lethal complic pulmonari vein is...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [138]:
sample_df = new_df.sample(n=1000000, random_state=200)
sample_df.reset_index(drop=True)

,text,labels
0,discoveri imidazoleisoindol deriv potent ido i...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,phytotox polymetal mine wast southern tuscani ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,mir promot tumor growth inhibit fbxw nsclc mir...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,construct autophagi relat prognost risk signat...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,calibr blood pressur measur jackson heart stud...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
999995,implement rehabilit model program inclus care ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
999996,effect mood stabil lithium noradrenerg turnov ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
999997,factor contribut excess sleepi midwiv practic ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
999998,pneumococc diseas thailand review examin epide...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [139]:
sample_df = sample_df.reset_index(drop=True)
sample_df

,text,labels
0,discoveri imidazoleisoindol deriv potent ido i...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,phytotox polymetal mine wast southern tuscani ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,mir promot tumor growth inhibit fbxw nsclc mir...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,construct autophagi relat prognost risk signat...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,calibr blood pressur measur jackson heart stud...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
999995,implement rehabilit model program inclus care ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
999996,effect mood stabil lithium noradrenerg turnov ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
999997,factor contribut excess sleepi midwiv practic ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
999998,pneumococc diseas thailand review examin epide...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [140]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [164]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 256
VALID_BATCH_SIZE = 256
EPOCHS = 20
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [165]:
train_size = 0.8
train_data=sample_df.sample(frac=train_size,random_state=200)
test_data=sample_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (2875804, 2)
TRAIN Dataset: (800000, 2)
TEST Dataset: (200000, 2)


In [166]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 8
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 8
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [167]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 124)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_featu

In [168]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [169]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [170]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%100==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
            torch.save(model.state_dict(), 'model.mod')
        loss.backward()
        optimizer.step()

In [171]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [172]:
# ONLY NEEDED ON FIRST RUN
#torch.save(model.state_dict(), 'model.mod')

In [173]:
import torch
torch.__version__

'1.13.0+cu116'

In [ ]:

model.load_state_dict(torch.load('model.mod'))
for epoch in range(EPOCHS):
   
    ###
    outputs, targets = validation(testing_loader)
    final_outputs = np.array(outputs) >=0.5
    val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
    val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

    print(f"Hamming Score = {val_hamming_score}")
    print(f"Hamming Loss = {val_hamming_loss}")
    ###
    train(epoch)
    

782it [05:50,  2.23it/s]


Hamming Score = 0.32572482184482177
Hamming Loss = 0.016845201612903225


0it [00:00, ?it/s]

Epoch: 0, Loss:  0.04016943648457527


100it [02:03,  1.22s/it]

...

500it [10:16,  1.22s/it]

Epoch: 4, Loss:  0.036497682332992554


600it [12:19,  1.22s/it]

Epoch: 4, Loss:  0.03719379007816315


612it [12:35,  1.23s/it]

In [ ]:
outputs, targets = validation(testing_loader)



In [ ]:
final_outputs = np.array(outputs) >=0.5

In [ ]:
for ii in range(150):
    ind_tar = [i for i, x in enumerate(targets[ii]) if x]
    print(ind_tar)
    ind_pred = [i for i, x in enumerate(final_outputs[ii]) if x]
    print(ind_pred)
    print()

In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")